In [1]:
# Pytorch version loss
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

class TorchCrossEntropy2d(nn.Module):

    def __init__(self, size_average=True, ignore_label=255):
        super(TorchCrossEntropy2d, self).__init__()
        self.size_average = size_average
        self.ignore_label = ignore_label

    def forward(self, predict, target, weight=None):
        """
            Args:
                predict:(n, c, h, w)
                target:(n, h, w)
                weight (Tensor, optional): a manual rescaling weight given to each class.
                                           If given, has to be a Tensor of size "nclasses"
        """
        assert not target.requires_grad
        assert predict.dim() == 4
        assert target.dim() == 3
        assert predict.size(0) == target.size(0), "{0} vs {1} ".format(predict.size(0), target.size(0))
        assert predict.size(2) == target.size(1), "{0} vs {1} ".format(predict.size(2), target.size(1))
        assert predict.size(3) == target.size(2), "{0} vs {1} ".format(predict.size(3), target.size(3))
        n, c, h, w = predict.size()
        target_mask = (target >= 0) * (target != self.ignore_label)
        target = target[target_mask]
        if not target.data.dim():
            return Variable(torch.zeros(1))
        predict = predict.transpose(1, 2).transpose(2, 3).contiguous()
        predict = predict[target_mask.view(n, h, w, 1).repeat(1, 1, 1, c)].view(-1, c)
        loss = F.cross_entropy(predict, target, weight=weight, size_average=self.size_average)
        return loss


class TorchBCEWithLogitsLoss2d(nn.Module):

    def __init__(self, size_average=True, ignore_label=255):
        super(TorchBCEWithLogitsLoss2d, self).__init__()
        self.size_average = size_average
        self.ignore_label = ignore_label

    def forward(self, predict, target, weight=None):
        """
            Args:
                predict:(n, 1, h, w)
                target:(n, 1, h, w)
                weight (Tensor, optional): a manual rescaling weight given to each class.
                                           If given, has to be a Tensor of size "nclasses"
        """
        assert not target.requires_grad
        assert predict.dim() == 4
        assert target.dim() == 4
        assert predict.size(0) == target.size(0), "{0} vs {1} ".format(predict.size(0), target.size(0))
        assert predict.size(2) == target.size(2), "{0} vs {1} ".format(predict.size(2), target.size(2))
        assert predict.size(3) == target.size(3), "{0} vs {1} ".format(predict.size(3), target.size(3))
        n, c, h, w = predict.size()
        target_mask = (target >= 0) * (target != self.ignore_label)
        target = target[target_mask]
        if not target.data.dim():
            return Variable(torch.zeros(1))
        predict = predict[target_mask]
        loss = F.binary_cross_entropy_with_logits(predict, target, weight=weight, size_average=self.size_average)
        return loss

In [40]:
def test_torch_cross_entropy_2d():
    # Create an instance of CrossEntropy2d
    criterion = TorchCrossEntropy2d()

    n, c, h, w = 2, 3, 4, 4
    predict = torch.tensor([
    [[[1, 2, 3, 4],
      [5, 6, 7, 8],
      [9, 10, 11, 12],
      [13, 14, 15, 16]],
     [[17, 18, 19, 20],
      [21, 22, 23, 24],
      [25, 26, 27, 28],
      [29, 30, 31, 32]],
     [[33, 34, 35, 36],
      [37, 38, 39, 40],
      [41, 42, 43, 44],
      [45, 46, 47, 48]]],
    [[[49, 50, 51, 52], 
      [53, 54, 55, 56],
      [57, 58, 59, 60],
      [61, 62, 63, 64]],
     [[65, 66, 67, 68],
      [69, 70, 71, 72],
      [73, 74, 75, 76],
      [77, 78, 79, 80]],
     [[81, 82, 83, 84],
      [85, 86, 87, 88],
      [89, 90, 91, 92],
      [93, 94, 95, 96]]]
    ], dtype=torch.float32)

    target = torch.tensor([
    [[1, 0, 2, 0],
     [2, 1, 0, 2],
     [0, 1, 2, 1],
     [2, 0, 1, 0]],
    [[0, 1, 2, 1],
     [1, 0, 2, 0],
     [2, 1, 0, 2],
     [0, 2, 1, 2]]
    ], dtype=torch.long)

    print(f'predict {predict.shape}')
    print(f'target {target.shape}')

    loss_custom = criterion(predict, target)
    loss_builtin = nn.CrossEntropyLoss()(predict, target)
    print(f'loss {loss_builtin}')
    
    assert torch.allclose(loss_custom, loss_builtin)

def test_torch_bce_with_logits_loss_2d():
    # Create an instance of CrossEntropy2d
    criterion = TorchBCEWithLogitsLoss2d()

    n, c, h, w = 2, 1, 4, 4
    predict = torch.tensor([
    [[[1, 2, 3, 4],
      [5, 6, 7, 8],
      [9, 10, 11, 12],
      [13, 14, 15, 16]]],
    [[[49, 50, 51, 52], 
      [53, 54, 55, 56],
      [57, 58, 59, 60],
      [61, 62, 63, 64]]]
    ], dtype=torch.float32)
    target = torch.tensor([[[[2, 1, 3, 1],
                         [0, 1, 0, 1],
                         [0, 1, 4, 1],
                         [0, 1, 5, 1]]],
                       [[[1, 6, 1, 0],
                         [1, 0, 7, 0],
                         [12, 0, 1, 0],
                         [1, 1, 1, 0]]]], dtype=torch.float32)
    print(f'predict {predict.shape}')
    print(f'target {target.shape}')


    loss_custom = criterion(predict, target)
    loss_builtin = nn.BCEWithLogitsLoss()(predict, target)
    print(f'loss {loss_builtin}')
    
    assert torch.allclose(loss_custom, loss_builtin)



In [36]:
test_torch_cross_entropy_2d()

predict torch.Size([2, 3, 4, 4])
target torch.Size([2, 4, 4])
loss 16.0


/opt/anaconda3/envs/csci1470/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [41]:
test_torch_bce_with_logits_loss_2d()

predict torch.Size([2, 1, 4, 4])
target torch.Size([2, 1, 4, 4])
loss -29.01507568359375


In [52]:
import tensorflow as tf
import numpy as np

from loss import *
def test_cross_entropy_2d():
    torch_loss = TorchCrossEntropy2d()
    tf_loss = CrossEntropy2d()

    # Define exact values for predict and target tensors
    n, c, h, w = 2, 3, 4, 4
    predict = torch.tensor([
    [[[1, 2, 3, 4],
      [5, 6, 7, 8],
      [9, 10, 11, 12],
      [13, 14, 15, 16]],
     [[17, 18, 19, 20],
      [21, 22, 23, 24],
      [25, 26, 27, 28],
      [29, 30, 31, 32]],
     [[33, 34, 35, 36],
      [37, 38, 39, 40],
      [41, 42, 43, 44],
      [45, 46, 47, 48]]],
    [[[49, 50, 51, 52], 
      [53, 54, 55, 56],
      [57, 58, 59, 60],
      [61, 62, 63, 64]],
     [[65, 66, 67, 68],
      [69, 70, 71, 72],
      [73, 74, 75, 76],
      [77, 78, 79, 80]],
     [[81, 82, 83, 84],
      [85, 86, 87, 88],
      [89, 90, 91, 92],
      [93, 94, 95, 96]]]
    ], dtype=torch.float32)

    target = torch.tensor([
    [[1, 0, 2, 0],
     [2, 1, 0, 2],
     [0, 1, 2, 1],
     [2, 0, 1, 0]],
    [[0, 1, 2, 1],
     [1, 0, 2, 0],
     [2, 1, 0, 2],
     [0, 2, 1, 2]]
    ], dtype=torch.long)
    print(f'predict {predict.shape}')
    print(f'target {target.shape}')

    predict_np = np.array([
    [[[1, 2, 3, 4],
      [5, 6, 7, 8],
      [9, 10, 11, 12],
      [13, 14, 15, 16]],
     [[17, 18, 19, 20],
      [21, 22, 23, 24],
      [25, 26, 27, 28],
      [29, 30, 31, 32]],
     [[33, 34, 35, 36],
      [37, 38, 39, 40],
      [41, 42, 43, 44],
      [45, 46, 47, 48]]],
    [[[49, 50, 51, 52],
      [53, 54, 55, 56],
      [57, 58, 59, 60],
      [61, 62, 63, 64]],
     [[65, 66, 67, 68],
      [69, 70, 71, 72],
      [73, 74, 75, 76],
      [77, 78, 79, 80]],
     [[81, 82, 83, 84],
      [85, 86, 87, 88],
      [89, 90, 91, 92],
      [93, 94, 95, 96]]]
    ], dtype=np.float32)

    target_np = np.array([
      [[1, 0, 2, 0],
      [2, 1, 0, 2],
      [0, 1, 2, 1],
      [2, 0, 1, 0]],
      [[0, 1, 2, 1],
      [1, 0, 2, 0],
      [2, 1, 0, 2],
      [0, 2, 1, 2]]
  ], dtype=np.int64)

    # Convert to TensorFlow tensors
    tf_predict = tf.constant(predict_np)
    tf_target = tf.constant(target_np)

    torch_res = torch_loss(predict, target)
    tf_res = tf_loss(tf_predict, tf_target)

    np.testing.assert_allclose(torch_res.numpy(), tf_res.numpy())


In [51]:
test_cross_entropy_2d()

predict torch.Size([2, 3, 4, 4])
target torch.Size([2, 4, 4])
torch loss 16.0
tf loss 16.0


In [55]:
import tensorflow as tf
import numpy as np

from loss import *
def test_bce_with_logits_lss_2d():
    torch_loss = TorchBCEWithLogitsLoss2d()
    tf_loss = BCEWithLogitsLoss2d()

    # Define exact values for predict and target tensors
    n, c, h, w = 2, 3, 4, 4
    predict = torch.tensor([
    [[[1, 2, 3, 4],
      [5, 6, 7, 8],
      [9, 10, 11, 12],
      [13, 14, 15, 16]]],
    [[[49, 50, 51, 52], 
      [53, 54, 55, 56],
      [57, 58, 59, 60],
      [61, 62, 63, 64]]]
    ], dtype=torch.float32)
    target = torch.tensor([[[[2, 1, 3, 1],
                         [0, 1, 0, 1],
                         [0, 1, 4, 1],
                         [0, 1, 5, 1]]],
                       [[[1, 6, 1, 0],
                         [1, 0, 7, 0],
                         [12, 0, 1, 0],
                         [1, 1, 1, 0]]]], dtype=torch.float32)
    print(f'predict {predict.shape}')
    print(f'target {target.shape}')

    predict_np = np.array([
    [[[1, 2, 3, 4],
      [5, 6, 7, 8],
      [9, 10, 11, 12],
      [13, 14, 15, 16]]],
    [[[49, 50, 51, 52],
      [53, 54, 55, 56],
      [57, 58, 59, 60],
      [61, 62, 63, 64]]]
    ], dtype=np.float32)

    target_np = np.array([[[[2, 1, 3, 1],
                            [0, 1, 0, 1],
                            [0, 1, 4, 1],
                            [0, 1, 5, 1]]],
                          [[[1, 6, 1, 0],
                            [1, 0, 7, 0],
                            [12, 0, 1, 0],
                        [1, 1, 1, 0]]]], dtype=np.float32)

    # Convert to TensorFlow tensors
    tf_predict = tf.constant(predict_np)
    tf_target = tf.constant(target_np)

    torch_res = torch_loss(predict, target)
    tf_res = tf_loss(tf_predict, tf_target)
    np.testing.assert_allclose(torch_res.numpy(), tf_res.numpy())


In [56]:
test_bce_with_logits_lss_2d()

predict torch.Size([2, 1, 4, 4])
target torch.Size([2, 1, 4, 4])
torch loss -29.01507568359375, tf loss -29.015077590942383


/opt/anaconda3/envs/csci1470/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
